In [8]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import torch

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# https://github.com/DmitryUlyanov/Multicore-TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE

from lda2vec_loss import loss, topic_embedding

ImportError: attempted relative import with no known parent package

In [2]:
def softmax(x):
    # x has shape [batch_size, n_classes]
    e = np.exp(x)
    n = np.sum(e, 1, keepdims=True)
    return e/n

# Load data

In [11]:
# dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
# docs = dataset['data']

# store each document with an initial id
# docs = [(i, doc) for i, doc in enumerate(docs)]

# "integer -> word" decoder 
decoder = np.load('decoder.npy')[()]

# for restoring document ids, "id used while training -> initial id"
doc_decoder = np.load('doc_decoder.npy')[()]

In [12]:
# original document categories
# targets = dataset['target']
# target_names = dataset['target_names']
# targets = np.array([targets[doc_decoder[i]] for i in range(len(doc_decoder))])

# Load the trained model

In [3]:
test_data = np.load('../stance/test_data.npy')

model = loss(*args, **kwargs)

state = torch.load('model_state.pytorch', map_location=lambda storage, loc: storage)
n_topics = 25

doc_weights = state['doc_weights.weight'].cpu().clone().numpy()
topic_vectors = state['topics.topic_vectors'].cpu().clone().numpy()
resulted_word_vectors = state['neg.embedding.weight'].cpu().clone().numpy()

# distribution over the topics for each document
topic_dist = softmax(doc_weights)

# vector representation of the documents
doc_vecs = np.matmul(topic_dist, topic_vectors)

NameError: name 'loss' is not defined

# Show topics

In [43]:
similarity = np.matmul(topic_vectors, resulted_word_vectors.T)
most_similar = similarity.argsort(axis=1)[:, -30:]

In [54]:
msl = list(map((lambda x: set(x)), most_similar.tolist()))
ints = set.intersection(*msl)

In [55]:
ints

{0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 16, 18, 20, 22, 23, 34}

In [57]:
most_similar_n = list(map((lambda x: list(x - ints)), msl))

In [58]:
most_similar_n

[[35, 6, 40, 15, 49, 21, 53, 25, 26, 30, 31],
 [64, 33, 35, 6, 40, 12, 15, 19, 21, 25, 30],
 [64, 35, 6, 43, 12, 17, 19, 21, 54, 25, 27],
 [33, 6, 40, 12, 15, 17, 19, 21, 25, 26, 28],
 [33, 65, 35, 6, 40, 15, 48, 19, 21, 24, 31],
 [35, 6, 40, 12, 15, 19, 21, 24, 25, 26, 28],
 [35, 6, 40, 12, 15, 48, 19, 21, 25, 30, 31],
 [32, 65, 35, 40, 12, 45, 46, 15, 82, 21, 30],
 [37, 6, 40, 74, 12, 50, 19, 21, 26, 28, 30],
 [33, 35, 37, 40, 15, 21, 25, 26, 28, 30, 31],
 [35, 37, 6, 12, 15, 48, 17, 19, 21, 25, 27],
 [33, 37, 6, 40, 15, 50, 19, 21, 25, 28, 30],
 [35, 58, 40, 42, 15, 21, 53, 24, 26, 30, 31],
 [64, 35, 6, 40, 43, 12, 15, 19, 21, 54, 25],
 [6, 40, 12, 17, 50, 19, 21, 25, 26, 28, 29],
 [35, 6, 40, 43, 12, 15, 17, 19, 21, 25, 26],
 [65, 35, 40, 12, 15, 17, 21, 24, 57, 26, 28],
 [35, 6, 12, 15, 48, 51, 24, 25, 27, 30, 31],
 [64, 35, 6, 40, 43, 12, 15, 19, 21, 24, 25],
 [35, 6, 15, 48, 50, 51, 21, 24, 26, 30, 31],
 [33, 6, 40, 12, 17, 50, 19, 21, 24, 25, 28],
 [33, 69, 40, 12, 17, 50, 19, 

In [60]:
# similarity = np.matmul(topic_vectors, resulted_word_vectors.T)
# most_similar = similarity.argsort(axis=1)[:, -10:]

for j in range(n_topics):
#     topic_words_list = 
    topic_words = ' '.join([decoder[i] for i in reversed(most_similar_n[j])])
    print('topic', j + 1, ':', topic_words)

topic 1 : candidate run time comment presidential good new republican come nbc tell
topic 2 : run comment good mexico republican immigrant come nbc tell love dump
topic 3 : immigration comment cut good mexico illegal immigrant tie nbc tell dump
topic 4 : american time comment good mexico illegal republican immigrant come nbc love
topic 5 : candidate right good mexico speak republican come nbc tell hair love
topic 6 : american time comment right good mexico republican immigrant come nbc tell
topic 7 : candidate run comment good mexico speak republican immigrant come nbc tell
topic 8 : run good week republican support look immigrant come tell hair word
topic 9 : run american time good mexico country immigrant big come nbc great
topic 10 : candidate run american time comment good republican come great tell love
topic 11 : immigration comment good mexico illegal speak republican immigrant nbc great tell
topic 12 : run american comment good mexico country republican come nbc great love
topi

# Show learned document embeddings

In [15]:
tsne = TSNE(perplexity=200, n_jobs=4)
X = tsne.fit_transform(doc_vecs.astype('float64'))

In [16]:
def plot(X):
    # X has shape [n_documents, 2]
    
    plt.figure(figsize=(16, 9), dpi=120);
    cmap = plt.cm.tab20
    number_of_targets = 20
    
    for i in range(number_of_targets):
        
        label = target_names[i]
        size = 15.0
        linewidths = 0.5
        edgecolors = 'k'
        color = cmap(i)
        
        if 'comp' in label:
            marker = 'x'
        elif 'sport' in label:
            marker = 's'
            edgecolors = 'b'
        elif 'politics' in label:
            marker = 'o'
            edgecolors = 'g'
        elif 'religion' in label:
            marker = 'P'
            size = 17.0
        elif 'sci' in label:
            marker = 'o'
            size = 14.0
            edgecolors = 'k'
            linewidths = 1.0
        elif 'atheism' in label:
            marker = 'P'
            size = 18.0
            edgecolors = 'r'
            linewidths = 0.5
        else:
            marker = 'v'
            edgecolors = 'm'
        
        plt.scatter(
            X[targets == i, 0], 
            X[targets == i, 1], 
            s=size, c=color, marker=marker,
            linewidths=linewidths, edgecolors=edgecolors,
            label=label
        );
    
    leg = plt.legend()
    leg.get_frame().set_alpha(0.3)

In [32]:
plot(X)  # learned document vectors

# different colors and markers represent 
# ground truth labels of each document

# open this image in new tab to see it better

IndexError: boolean index did not match indexed array along dimension 0; dimension is 46878 but corresponding boolean dimension is 18846

<Figure size 1920x1080 with 0 Axes>

# Show initial document weights (vanilla lda)

In [33]:
doc_weights_init = np.load('doc_weights_init.npy')

In [34]:
tsne = TSNE(perplexity=200, n_jobs=4)
Y = tsne.fit_transform(doc_weights_init.astype('float64'))

In [ ]:
# to initialize topic assignments for lda2vec algorithm 
# I run normal LDA and used output distributions over topics 
# of each document

plot(Y)  # distribution over the topics for each document (output of LDA)

# different colors and markers represent 
# ground truth labels of each document

# open this image in new tab to see it better

# Explore learned topic distributions

In [ ]:
tsne = TSNE(perplexity=200, n_jobs=4)
Z = tsne.fit_transform(topic_dist.astype('float64'))

In [ ]:
plot(Z)  # learned distribution over the topics for each document

# these are topic assignments as on the plot above 
# but these ones are after the training of lda2vec

# different colors and markers represent 
# ground truth labels of each document

# open this image in new tab to see it better

In [ ]:
# distribution of nonzero probabilities
dist = topic_dist.reshape(-1)
plt.hist(dist[dist > 0.01], bins=40);

In [ ]:
# distribution of probabilities for some random topic
plt.hist(topic_dist[:, 10], bins=40);

In [ ]:
# topic assignments for two random topics
plt.scatter(topic_dist[:, 10], topic_dist[:, 20]);

In [ ]:
# correlation of topic assignments
corr = np.corrcoef(topic_dist.transpose(1, 0))
plt.imshow(corr);
plt.colorbar();

# Show a document and its topics

In [ ]:
i = 100 # document id

print('DOCUMENT:')
print([doc for j, doc in docs if j == doc_decoder[i]][0], '\n')

print('DISTRIBUTION OVER TOPICS:')
s = ''
for j, p in enumerate(topic_dist[i], 1):
    s += '{0}:{1:.3f}  '.format(j, p)
    if j%6 == 0:
        s += '\n'
print(s)

print('\nTOP TOPICS:')
for j in reversed(topic_dist[i].argsort()[-3:]):
    topic_words = ' '.join([decoder[i] for i in reversed(most_similar[j])])
    print('topic', j + 1, ':', topic_words)